## Data clean / exploration

In [2]:
import csv
import pandas as pd
import numpy as np

import seaborn as sns

In [3]:
# import data
calendar_all = pd.read_csv('./data/calendar.csv')
listings_all = pd.read_csv('./data/listings.csv')
sentiment_scores = pd.read_csv('./data/scores.csv')
# reviews = pd.read_csv('./data/reviews.csv')


In [4]:
# do a bit of cleaning

# remove cols that only contain (1) unique elem
listings = listings_all
for col in listings_all.columns:
    if len(listings_all[col].unique()) == 1:
        listings = listings.drop(col,axis=1)

In [5]:
list(listings.columns.values)

listings.host_is_superhost = listings['host_is_superhost'].replace({'f': 0, 't': 1})

In [6]:
# Trimmed Frame
trimmed_listings = listings[['id','host_id', 'host_response_rate',
                             'host_is_superhost', 'neighbourhood_group_cleansed', 'property_type', 'room_type', 'accommodates',
                             'guests_included', 'bathrooms', 'bedrooms', 'beds', 'price',
                               'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
                               'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
                               'review_scores_location', 'review_scores_value']]

In [7]:
# populate review NA's with average
pd.options.mode.chained_assignment = None 
trimmed_listings['review_scores_rating'] = trimmed_listings.review_scores_rating.fillna(trimmed_listings.review_scores_rating.median())
trimmed_listings['review_scores_accuracy'] = trimmed_listings.review_scores_accuracy.fillna(trimmed_listings.review_scores_accuracy.median())
trimmed_listings['review_scores_cleanliness'] = trimmed_listings.review_scores_cleanliness.fillna(trimmed_listings.review_scores_cleanliness.median())
trimmed_listings['review_scores_checkin'] = trimmed_listings.review_scores_checkin.fillna(trimmed_listings.review_scores_checkin.median())
trimmed_listings['review_scores_communication'] = trimmed_listings.review_scores_communication.fillna(trimmed_listings.review_scores_communication.median())
trimmed_listings['review_scores_location'] = trimmed_listings.review_scores_location.fillna(trimmed_listings.review_scores_location.median())
trimmed_listings['review_scores_value'] = trimmed_listings.review_scores_value.fillna(trimmed_listings.review_scores_value.median())


In [8]:
# Add sentiment scores column
# Used Microsoft Azure Sentiment Text Analysis API through Postman requests to identify the sentiment score 
#  of listing descriptions from 1 - 100
trimmed_listings = trimmed_listings.join(sentiment_scores['score'], how="inner")

# rename to something better
trimmed_listings.rename(index=str, columns={"neighbourhood_group_cleansed": "neighbourhood"}, inplace=True)


In [9]:
trimmed_listings['price'] = trimmed_listings['price'].replace('[\$\,\.]', '', regex=True).astype(int) / 100

In [10]:
# change percentage strings to ints
trimmed_listings['host_response_rate'] = trimmed_listings['host_response_rate'].str.replace('%', '')
trimmed_listings['host_response_rate'] = trimmed_listings['host_response_rate'].fillna('-1')
trimmed_listings['host_response_rate'] = trimmed_listings['host_response_rate'].astype(int)
trimmed_listings['host_response_rate'] = trimmed_listings['host_response_rate'].replace(-1, np.nan)
trimmed_listings['host_response_rate'] = trimmed_listings.host_response_rate.fillna(trimmed_listings.host_response_rate.mean())

In [11]:
# drop few records with no room information
trimmed_listings= trimmed_listings[pd.notnull(trimmed_listings['host_is_superhost'])]
trimmed_listings= trimmed_listings[pd.notnull(trimmed_listings['beds'])]
trimmed_listings= trimmed_listings[pd.notnull(trimmed_listings['bathrooms'])]
trimmed_listings= trimmed_listings[pd.notnull(trimmed_listings['bedrooms'])]

In [12]:
# remove infrequent property types
trimmed_listings = trimmed_listings[trimmed_listings.property_type.isin(['Apartment', 'House' , 'Condominium',
       'Townhouse', 'Loft', 'Bed & Breakfast'])]


cleaned_base = trimmed_listings

In [13]:
# convert categories to dummies
trimmed_listings = pd.get_dummies(trimmed_listings)

In [14]:
# Rename features with spaces in their names
trimmed_listings.rename(index=str, columns={
    'neighbourhood_Beacon Hill': 'neighbourhood_Beacon_Hill',
    'neighbourhood_Capitol Hill': 'neighbourhood_Capitol_Hill',
    'neighbourhood_Central Area': 'neighbourhood_Central_Area',
    'neighbourhood_Lake City': 'neighbourhood_Lake_City',
    'neighbourhood_Other neighborhoods': 'neighbourhood_Other_neighborhoods',
    'neighbourhood_Queen Anne': 'neighbourhood_Queen_Anne',
    'neighbourhood_Rainier Valley': 'neighbourhood_Rainier_Valley',
    'neighbourhood_Seward Park': 'neighbourhood_Seward_Park',
    'neighbourhood_University District': 'neighbourhood_University_District',
    'neighbourhood_West Seattle': 'neighbourhood_West_Seattle',
    'property_type_Bed & Breakfast': 'property_type_Bed_Breakfast',
    'room_type_Entire home/apt': 'room_type_Entire_home_apt',
    'room_type_Private room': 'room_type_Private_room',
    'room_type_Shared room': 'room_type_Shared_room'
}, inplace=True)


In [15]:
trimmed_listings.to_csv("./data/cleaned-listings-dummies.csv", index = False)
cleaned_base.to_csv("./data/cleaned-listings-base.csv", index = False)
